<figure>
  <img src="../images/tudelft_logo.png" alt="image" width="250" align="right"/>
</figure>

# 5: Coastal Classification and Beach States

Welcome to the notebook of Week 5! With this notebook, we focus on morphodynamics of the upper shoreface, supplementing the material treated in Chapter 7 of the [Coastal Dynamics Open Textbook](https://books.open.tudelft.nl/home/catalog/book/202). The main topics are coastal classification (see also Chapter 2 and 4 in the book) and beach states. As you may remember, we have explored coastal classification before, not only in the Week 1 notebooks but also in the Week 3 notebook on tidal environments. However, we feel that this topic is especially important to revisit before considering the beach states, to better connect your knowledge with Chapter 7 of this course. 

As you know from Chapter 2 and 4 of the book and the 1a, 1b and 3a notebooks, coasts can be classified based on tectonics and physical processes. In this notebook, we will only consider process-based classification. We will look at data and satellite imagery of different Brazilian beaches, and try to classify them in various ways, considering the effects of both waves and tides. 

The notebook consists of the following three parts:
1. Exploration of the available data and satellite imagery (*3 multiple-choice questions and one text question*)
2. Classification based on relative wave and tide influence (*3 multiple-choice and 2 multiple-selection questions*)
3. Beach states (*12 multiple-choice questions and 1 multiple-selection question*)


## Import the required libraries, questions and data.

Run the below three cells to import:
1. All libraries that we use for our analysis
2. The question files
3. The dataset with data for different coordinates along the Brazilian coast 

In [ ]:
from pathlib import Path
import warnings

import bokeh.io
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import numpy as np
import pandas as pd
import panel as pn
from bokeh.resources import INLINE

import coastal_dynamics as cd

# Activate Panel extension to make interactive visualizations
pn.extension()

In [ ]:
import sys

sys.path.append('../')

from modules import mod_5

In [ ]:
# Read questions from local file
questions = cd.read_questions(Path("../hashed_questions/5_coastal_impact_beach_states_hashed.json"))

question_industry = cd.QuestionIndustry(questions)

cd.UseAnswersApp("5").serve()

In [ ]:
# file path
fp = Path("../database/5_coastal_impact_beach_states/5_data.csv")

# We load this file as a Dataframe
df = pd.read_csv(fp, sep=";", encoding="latin1", engine="python", decimal=",")

# We add a column containing the geometry of each entry using geopandas. For this dataset, these are points (longitude, latitude)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:4326")

# uncomment to show the (geo)dataframe
gdf           

## Part 1: Get to know the data

### Aggregated data set

The dataframe (see above) shows a total of 13 beaches from 4 different Brazilian states. These states (from north to south) are Pará, Maranhão, Rio de Janeiro, and Santa Catarina. The dataset has been aggregated and adapted by us for the purposes of this notebook. 

We have based ourselves on:
* [Athanasiou et al. (2023)](https://zenodo.org/records/8200200)
* [Klein et al. (2016)](https://link.springer.com/book/10.1007/978-3-319-30394-9)
* [Dadalto et al. (2022)](https://www.cambridge.org/core/journals/quaternary-research/article/holocene-morphosedimentary-evolution-of-marambaia-barrier-island-se-brazil/675595DC02D7E9C841FD794F9F0061CB)

Specifically, the first source is used to get values for the significant wave height ($H_s$), the peak period ($T$) and the mean tidal range (MTR). The second source is used to get the relative tidal range RTR (see Eq. 4.2 in the book), the mean grain diameter $D$ and the beach slope for the Santa Catarina, Maranhão, and Pará states. The third source is used to get the RTR, the $D$ and the beach slope for the Rio de Janeiro state. Some of the values have been modified slightly for educational purposes. 

Using the data from the three sources, we computed the wave height at breaking $H_b$, the fall velocity $w_s$ and the dimensionless fall velocity $\Omega$ and added these to the dataframe. We will not use these computed parameters until Part 3, where we will further explain the parameters and how we computed them computations. 

We should note that for the Santa Catarina beaches, we had only regional values available for the sediment size, and the beach slope. Therefore, some of these beaches have equal values for the sediment size and beach slope. 

The regions are displayed in the table below, so you can see which beaches belong to which regions. The region is also part of the above dataframe. 

| Region (in Santa Catarina) | Coastal section                  | D [mm] | beach slope [degrees] |
| :---------------------------------- | :------------------------------- | :----- | :------------------------ |
| 1                                   | Barra Velha, <br> Do Ubatuba     | 0\.300  | 5\.0                      |
| 2                                   | Balnario Camboriu, <br> Tijucas  | 0\.320  | 5\.0                      |
| 3                                   | Praia do Macambique              | 0\.300  | 4\.1                      |
| 4                                   | Enseada de Pinheira, <br> Laguna | 0\.190  | 3\.0                      |
| 5                                   | Campo Bom                        | 0\.210  | 1\.8                      |

### Warning!
At many points in your career, you may have to assess coastal sites based on limited information. Learning to critically interpret datasets aggregated from multiple sources (like this one), and especially to critically assess the accuracy of "global" datasets at a local scale is an essential skill for the modern coastal engineer. Athanasiou et al. (2023) for example use a numerical model to compute offshore wave data for sites around the world, whereas Klein et al. (2016) use direct measurements from Brazil. All of this is to say that this data is not presumed to be locally accurate. However, in this context, we use it to compare the Brazilian coastline on a larger regional scale and draw broad conclusions to illustrate the concepts introduced in this course. The $\Omega$ values as presented in the dataset should be sufficient for qualitatively comparing the different beaches.

Nonetheless, we would like to stress that the values should not be interpreted as highly accurate for specific beaches, given the coarse resolution and simplifying assumptions of the global dataset and limited local data. However, they are still useful to get a feel for the surrounding area (and relative differences from other regions), and to get first estimates of the order of magnitude for these parameters.

### Brazilian beaches panel

We are now going to create a plot that shows satellite images of the above listed beaches in a separate panel, the Brazilian beaches panel! 

In this Brazilian beaches panel, satellite images and data from the selected beaches from the Brazilian states of (from north to south) Pará, Maranhão, Rio de Janeiro, and Santa Catarina are shown:
- You can select one of the 13 beaches, grouped by state.
- You can freely zoom and move around in the presented satellite image.
- You can see all of the values from the dataframe when hovering over a data point with the cursor. Some of these values will be needed to answer the questions below.
- Please note that altering the slider positions or selecting different options from the dropdown menus may trigger a warning; it can safely be ignored, and possibly silenced by adjusting the logging warning level. 

Go ahead and run the below code cell!

In [ ]:
app = mod_5.plot_brazilian_coast(gdf)

cd.launch_app(app)

### Questions about Part 1

Now, using the Brazilian beaches panel, try to answer the questions below. For each question, try to relate your answer to the satellite imagery. 

**Note**: Remember that we are working with an aggregated dataset. Each data point should be interpreted as representative of the region around it, and not as specific to that beach. How well do the provided data match what you see in the satellite images? What clues can you look for to check whether the data look reasonable?

In [ ]:
q = [
    "Q5a-highest_MTR",
    "Q5a-highest_Hs",
    "Q5a-Marambaia_barrier",
    "Q5a-alongshore_transport",
]

question_industry.serve(q)

## Part 2: Classification based on relative wave and tide influence

The Brazilian coast has tides ranging from 0.5 to 11 m and waves from low to high. We can therefore find very diverse beaches. In Part 1, you have seen that the beaches in the Southern states are wave-dominated and that the influence of tides becomes significant for the Northern states. Let's look into this a bit further. 

There a a few methods to classify coastal systems and beaches based on the relative influence of waves and tides; have a look at Chapter 4.4.2 in the textbook. Coasts can be classified into three types based on relative tide range RTR, as described above (see Eq. 4.2 in the book). This parameter quantifies the relative contribution of waves and tides. We can also investigate the relative influence of waves and tides by using a figure as Figure 4.13. Let's try to classify some of the coastal sections using the mean tidal range (MTR) and mean significant wave height ($H_s$)! We plot some of the Brazilian coasts and see how they are classified using the figure. 

Run the below cell to generate this plot. 

**Note**: Again, you can hover over points to get relevant values. You can also highlight points by clicking the labels in the legend.

In [ ]:
mod_5.fig413(gdf)

### Questions about Part 2

Using the above figure, and the Brazilian beaches plot, answer the questions below. 

Don't forget to also have a look at the relative tidal ranges (RTR) for the different beaches! To do so, hover over points to get relevant values. highlight points by clicking the labels in the legend.

#### Some notes on the RTR classification
The maximum RTR for this specific dataset is quite small, but it can be much bigger (for instance values of 10-50). When waves are relatively high and tides low (RTR < 3) beaches are wave-dominated. When waves are very low and the tide relatively high (RTR larger then, say, 15) the coast becomes tide-dominated and gradually approaches the pure tidal flat situation. In the intermediate range, we will still have beaches shaped by waves, but they carry some distinct tidal characteristics. They are called tide-dominated beaches in this notebook, but you will also find the term *tide-modified* in literature. The RTR values for the selected Pará and São Luís beaches hint towards wave-shaped beaches with significant tidal influence.

In [ ]:
q = [
    "Q5a-Princesa_Tijucas",
    "Q5a-Ajuruteua",
    "Q5a-SaoLuis_DoUbatuba",
    "Q5a-fluvial_sediment",
    "Q5a-tombolo",
]

question_industry.serve(q)

## Part 3: Beach states

We will now explore the topic of beach states. We will focus on the wave-dominated coastal sections (i.e., the coastal sections from the Rio de Janeiro and Santa Catarina coastline). From Section 7.3 of the book, you know that a wave-dominated beach may be classified as reflective or dissipative, or an intermediate state in between. Note that there is a broad spectrum of intermediate beach states. In this notebook, we will not look at this in detail and make some broad generalizations. For instance, we call higher intermediate states (i.e.,  “more dissipative” and lower intermediate states “more reflective”. 

**Note** that the coastal sections considered as wave-dominated here all have a relative tidal range of lower than 3.

### Theory
A common classifier used besides the Iribarren number is the dimensionless fall velocity, which is calculated as follows (see Eq. 7.8 from the book): 
$$
\Omega = \frac{H_b}{w_s T}
$$
where $H_b$ is the wave height at breaking, $T$ is the wave period and $w_s$ is the fall velocity. 

$\Omega$ quantifies the relative contribution of wave height and period and sediment grain size (expressed as sediment fall velocity) to beach morphodynamics. When waves are relatively low, periods long and sand coarse ($\Omega < 1$), the beaches are narrow without bars and called reflective. When waves are moderate to high ($2 < \Omega < 5$), the beaches usually have one or two bars cut by rip channels and currents. When wave are high and sand is fine (($\Omega > 6$) the beaches become wide and dissipative with often multiple shore-parallel sand bars. A recent, very nice overview of beach states is shown in [Castelle and Masselink (2022)](https://doi.org/10.1017/cft.2022.2). 

### Computations 
Using the data from the three sources, we computed the wave height at breaking $H_b$, the fall velocity $w_s$ and the dimensionless fall velocity $\Omega$. These values are added to the loaded dataframe and to the satellite images, for you to inspect.

We computed the wave height at breaking $H_b$ as MTR/RTR, using Eq. 4.2 in the book and ignoring any differences between the mean tidal range and the mean spring tidal range. The fall velocity $w_s$ was calculated using Soulsby (1997). You don't have to remember this equation, but it is included here for completeness:

$$ w_s = \frac{\nu_{kin}}{D} \left( \sqrt{10.36^2 + 1.049 * (D_{*}^3)} - 10.36 \right) $$
with
$ D_{*} = \left(\frac{(\rho_s / \rho_w - 1)g}{\nu_{kin}^2}\right) ^{1/3} D $, $g=9.81$ m/s$^2$, $\nu_{kin}=1.36 \cdot 10^{-6}$ m$^2$/s, $\rho_w=1027$ kg/m$^3$, $\rho_s=2650$ kg/m$^3$

### Beach slope versus sediment size plot

Klein et al. (2005) use beach slope and sediment size as a proxy for classifying wave-dominated beaches with different exposure, see also Figure 7.10 in the book. Let's try this for ourselves! We include all 13 beaches, so not only the wave-dominated beaches but the tidally influenced ones as well.

As pointed out in Part 1, we had only regional values available for the sediment size, and the beach slope of the Santa Catarina beaches.  Therefore, some of these beaches have equal values for the sediment size and beach slope; they will therefore plot on top of each other in the below figure. This is also the likely reason that we don't really see any reflective beaches, at least with respect to the data. Note that different coastal sections might still have varying dimensionless fall velocity, even if they are within the same region (you can check the values from the dataframe or data with the satellite imagery). Think about why this is the case.

You can now run the below cell to get the plot. 

**Note**: Again, you can hover over points to get relevant values. You can remove or add locations by (de)selecting them using the checkboxes. Note that this may change the color of each point, which is updated accordingly in the legend.

In [ ]:
mod_5.fig710(df)

### Questions about Part 3

Using the figure above, as well as the Brazilian beaches panel with satellite imagery, try to answer the questions below.

In [ ]:
q = [
    "Q5b-grain_size_slope",
    "Q5b-beach_characteristics_error",
    "Q5b-lowest_dimless_fall_velocity",
    "Q5b-value_fall_velocity_CamboBom",
    "Q5b-number_of_bars",
    "Q5b-reflective_or_dissipative",
    "Q5b-beach_state",
    "Q5b-value_fall_velocity_EnseadaDePinheira",
    "Q5b-characteristics_reflective_beach1",
    "Q5b-characteristics_reflective_beach2",
    "Q5b-BarraVelha",
    "Q5b-compare_dimless_fall_velocity",
    "Q5b-coastal_appreciation",
]

question_industry.serve(q)

## Disclaimer

When using satellite images to investigate coastal sites, bear in mind several important limitations: the image captures only a snapshot in time, and may therefore not be representative of average or extreme conditions. For example, an image taken at low tide in calm conditions might give a very different impression from an image at high tide in rough conditions. This is further complicated by the frequent presence of clouds, especially in stormy weather. Usually studies analyzing remotely-sensed coastal systems average together images from multiple time periods to get a better idea of the system's behaviour. Furthermore, the imagery dataset used in this notebook is updated automatically from year to year, so some of the beach conditions described above may have changed since we developed it in March 2025. Remote sensing is an exciting new frontier for coastal engineering and geoscience, but as with any data source, it is important to ask critical questions about what you are looking at. 

## The end

You have reached the end of this notebook 5. This was the only notebook of this week.
